# ResNet 만들기



1.   ResNet은 스킵 구조를 이용한 CNN 신경망이다.
2.   **기울기 소실**은 은닉층이 깊어짐에 따라 입력층에 가까운 가중치들의 기울기가 0에 가까워지는 현상을 의미한다. 기울기가 0이 되면 가중치가 더 이상 업데이트되지 않기 때문에 학습이 이루어지지 않는다.
3. 배치 정규화는 배치 간의 차이를 정규화해주므로 더 안정되게 학습할 수 있다.
4. **스킵 커넥션은 은닉층을 거치지 않은 입력값과 은닉층의 결과를 더하는 구조**를 의미한다.
5. 평균 풀링은 커널의 평균값을 이용하는 풀링이다.



## ResNet 구조
https://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/He_Deep_Residual_Learning_CVPR_2016_paper.pdf



*   스킵 커넥션을 이용해 기울기 소실 문제 일부 해결





## 기본 블록 정의하기

In [1]:
import torch
import torch.nn as nn

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size = 3):
        super().__init__()
        #합성곱층 정의
        self.c1 = nn.Conv2d(in_channels, out_channels, kernel_size = kernel_size, padding=1)
        self.c2 = nn.Conv2d(out_channels, out_channels, kernel_size = kernel_size, padding=1)
        self.downsample = nn.Conv2d(in_channels, out_channels, kernel_size = 1)

        #배치 정규화층 정의
        self.bn1 = nn.BatchNorm2d(num_features=out_channels)
        self.bn2 = nn.BatchNorm2d(num_features=out_channels)

        self.relu = nn.ReLU()

    def forward(self, x):
        #스킵 커넥션을 위히 초기 입력 저장
        x_ = x


        x = self.c1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.c2(x)
        x = self.bn2(x)

        #합성곱의 결과와 입력의 채널 수를 맞춤
        x_ = self.downsample(x_)
        #합성곱층의 결과와 저정해놨던 입력값을 더해줌(스킵 커넥션)
        x += x_
        x = self.relu(x)

        return x


## ResNet 모델 정의하기

In [2]:
class ResNet(nn.Module):
    def __init__(self, num_classes = 10):
        super().__init__()
        #기본 블록
        self.b1 = BasicBlock(in_channels=3, out_channels=64)
        self.b2 = BasicBlock(in_channels=64, out_channels=128)
        self.b3 = BasicBlock(in_channels=128, out_channels=256)

        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)

        #분류기
        self.fc1 = nn.Linear(in_features=4096, out_features=2048)
        self.fc2 = nn.Linear(in_features=2048, out_features=512)
        self.fc3 = nn.Linear(in_features=512, out_features=num_classes)

        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.b1(x)
        x = self.pool(x)
        x = self.b2(x)
        x = self.pool(x)
        x = self.b3(x)
        x = self.pool(x)
        x = torch.flatten(x, start_dim = 1)

        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x


## 모델 학습

In [3]:
from torchvision.datasets.cifar import CIFAR10
from torchvision.transforms import Compose, ToTensor, RandomHorizontalFlip, RandomCrop, Normalize
from torch.utils.data.dataloader import DataLoader
from torch.optim.adam import Adam

#이미지 전처리 방법 정의
transforms = Compose([
    RandomCrop((32,32), padding=4),
    RandomHorizontalFlip(p=0.5),
    ToTensor(),
    Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.243, 0.261))
])

#데이터셋 정의
training_data = CIFAR10(root="./", train = True, download = True, transform = transforms)
test_data = CIFAR10(root="./", train = False, download = True, transform = transforms)

#데이터로더 정의
train_loader = DataLoader(training_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

100%|██████████| 170498071/170498071 [00:02<00:00, 57168310.60it/s]


Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified


In [4]:
#GPU 설정 및 모델 객체 생성
device = "cuda" if torch.cuda.is_available() else "cpu"

model = ResNet(num_classes=10)
model.to(device)

ResNet(
  (b1): BasicBlock(
    (c1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (c2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (downsample): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (b2): BasicBlock(
    (c1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (c2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (downsample): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (b3): BasicBlock(
    (c1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [5]:
import tqdm
#학습 루프
lr = 1e-4
optim = Adam(model.parameters(), lr = lr)

for epoch in range(30):
    iterator = tqdm.tqdm(train_loader)
    for data, label in iterator:
        #최적화를 위한 기울기 초기화
        optim.zero_grad()
        #모델의 예측값
        preds = model(data.to(device))
        #소실 계산 및 역전파
        loss = nn.CrossEntropyLoss()(preds, label.to(device))
        loss.backward()
        optim.step()

        iterator.set_description(f"epoch:{epoch+1} loss:{loss.item()}")

torch.save(model.state_dict(), "ResNet.pth")

epoch:30 loss:0.06888166815042496: 100%|██████████| 1563/1563 [00:47<00:00, 32.69it/s]


## 모델 성능 평가하기

In [6]:
model.load_state_dict(torch.load("ResNet.pth", map_location=device))

num_corr = 0

with torch.no_grad():
    for data, label in test_loader:
        output = model(data.to(device))
        preds = output.data.max(1)[1]
        corr = preds.eq(label.to(device).data).sum().item()
        num_corr += corr

    print(f"Accuracy : {num_corr/len(test_data)}")

Accuracy : 0.8766
